**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [ ]:
import os
import re
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import itertools
from itertools import chain, product
from itertools import zip_longest
import gensim
from gensim import corpora
from pprint import pprint
from gensim import models
from gensim.models import LdaModel
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.models import CoherenceModel
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import string
import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.manifold import TSNE

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Load Dataset**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Train.csv')
Test_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Test.csv')
Val_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Val.csv')

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 3), (2140, 3), (2140, 2))

In [ ]:
sentences = []

for i in Train_data['tweet']:
    sentences.append(i)
for j in Test_data['tweet']:
    sentences.append(j)
for k in Val_data['tweet']:
    sentences.append(k)

In [ ]:
#create new df 
df = pd.DataFrame({'Text': sentences})
df.head()

,Text
0,The CDC currently reports 99031 deaths. In gen...
1,States reported 1121 deaths a small rise from ...
2,Politically Correct Woman (Almost) Uses Pandem...
3,#IndiaFightsCorona: We have 1524 #COVID testin...
4,Populous states can generate large case counts...


In [ ]:
df.shape

(10700, 1)

In [ ]:
data_text = df[['Text']]
data_text['index'] = data_text.index
documents = data_text

In [ ]:
documents.head()

,Text,index
0,The CDC currently reports 99031 deaths. In gen...,0
1,States reported 1121 deaths a small rise from ...,1
2,Politically Correct Woman (Almost) Uses Pandem...,2
3,#IndiaFightsCorona: We have 1524 #COVID testin...,3
4,Populous states can generate large case counts...,4


In [ ]:
print(len(documents))
print(documents[:10])

10700
                                                Text  index
0  The CDC currently reports 99031 deaths. In gen...      0
1  States reported 1121 deaths a small rise from ...      1
2  Politically Correct Woman (Almost) Uses Pandem...      2
3  #IndiaFightsCorona: We have 1524 #COVID testin...      3
4  Populous states can generate large case counts...      4
5  Covid Act Now found "on average each person in...      5
6  If you tested positive for #COVID19 and have n...      6
7  Obama Calls Trump’s Coronavirus Response A Cha...      7
8  ???Clearly, the Obama administration did not l...      8
9  Retraction—Hydroxychloroquine or chloroquine w...      9


#**Data Pre-processing**

**Loading gensim and nltk libraries**

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from nltk import PorterStemmer
PorterStemmer().stem('complications')
from nltk.stem.porter import *
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


**Write a function to perform lemmatize and stem preprocessing steps on the data set.**

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

**Select a document to preview after preprocessing.**

In [ ]:
doc_sample = documents[documents['index'] == 10699].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Another', '4422', 'new', 'coronavirus', 'cases', 'have', 'been', 'confirmed', 'in', 'the', 'UK', '-', 'the', 'highest', 'daily', 'number', 'since', '8', 'May.', "It's", 'up', 'from', '4322', 'new', 'cases', 'reported', 'on', 'Friday', 'and', 'the', 'overall', 'total', 'nationwide', 'now', 'stands', 'at', '385936.', 'Read', 'the', 'latest', 'here:', 'https://t.co/1TYdoRZ9Yr', 'https://t.co/96F7njc7sM']


 tokenized and lemmatized document: 
['coronaviru', 'case', 'confirm', 'highest', 'daili', 'number', 'case', 'report', 'friday', 'overal', 'total', 'nationwid', 'stand', 'read', 'latest', 'http', 'tydorz', 'http']


**Preprocess the headline text, saving the results as ‘processed_docs'**

In [ ]:
#processed_docs = documents['headline_text'].map(preprocess)
processed_docs = documents['Text'].map(preprocess)
processed_docs[:10]

0    [current, report, death, gener, discrep, death...
1    [state, report, death, small, rise, tuesday, s...
2    [polit, correct, woman, use, pandem, excus, re...
3    [covid, test, laboratori, india, august, test,...
4    [popul, state, gener, larg, case, count, look,...
5    [covid, averag, person, illinoi, covid, infect...
6    [test, posit, covid, symptom, stay, home, away...
7    [obama, call, trump, coronaviru, respons, chao...
8    [clearli, obama, administr, leav, kind, game, ...
9    [retract, chloroquin, macrolid, treatment, cov...
Name: Text, dtype: object

**Bag of Words on the Data set**

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 count
1 current
2 death
3 differ
4 discrep
5 explic
6 gener
7 peopl
8 report
9 roughli
10 small


In [ ]:
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)

**Gensim doc2bow**

In [ ]:
#bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus[4310]
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[10699]

[(6, 1),
 (9, 1),
 (15, 1),
 (29, 2),
 (68, 1),
 (84, 1),
 (110, 1),
 (143, 1),
 (166, 1),
 (174, 1),
 (302, 1),
 (597, 1),
 (641, 1)]

**Preview Bag Of Words for our sample preprocessed document.**

In [ ]:
#bow_doc_4310 = bow_corpus[4310]
bow_doc_4310 = bow_corpus[10699]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 6 ("report") appears 1 time.
Word 9 ("stand") appears 1 time.
Word 15 ("coronaviru") appears 1 time.
Word 29 ("case") appears 2 time.
Word 68 ("daili") appears 1 time.
Word 84 ("read") appears 1 time.
Word 110 ("total") appears 1 time.
Word 143 ("latest") appears 1 time.
Word 166 ("confirm") appears 1 time.
Word 174 ("number") appears 1 time.
Word 302 ("highest") appears 1 time.
Word 597 ("overal") appears 1 time.
Word 641 ("nationwid") appears 1 time.


**TF-IDF**

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.31211130042837376),
 (1, 0.2378932858253197),
 (2, 0.43617516091149683),
 (3, 0.2815315786425528),
 (4, 0.30332798818827583),
 (5, 0.1428180480140585),
 (6, 0.13052669237651968),
 (7, 0.34238644777046995),
 (8, 0.3087730044317558),
 (9, 0.30560381027153183),
 (10, 0.17537958339798476),
 (11, 0.3321468112858137)]


**Running LDA using Bag of Words**

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.053*"covid" + 0.035*"test" + 0.019*"state" + 0.018*"coronaviru" + 0.015*"peopl" + 0.010*"countri" + 0.009*"restrict" + 0.009*"day" + 0.009*"say" + 0.009*"sampl"
Topic: 1 
Words: 0.067*"test" + 0.048*"report" + 0.034*"case" + 0.032*"state" + 0.028*"covid" + 0.023*"death" + 0.022*"number" + 0.019*"daili" + 0.018*"updat" + 0.017*"coronaviru"
Topic: 2 
Words: 0.044*"covid" + 0.020*"health" + 0.019*"state" + 0.017*"coronaviru" + 0.013*"case" + 0.011*"pradesh" + 0.010*"home" + 0.009*"report" + 0.008*"nation" + 0.007*"maharashtra"
Topic: 3 
Words: 0.104*"case" + 0.066*"covid" + 0.039*"confirm" + 0.026*"death" + 0.023*"state" + 0.021*"nigeria" + 0.020*"discharg" + 0.019*"lago" + 0.017*"activ" + 0.015*"report"
Topic: 4 
Words: 0.076*"coronaviru" + 0.032*"case" + 0.031*"test" + 0.030*"trump" + 0.027*"covid" + 0.024*"say" + 0.020*"presid" + 0.017*"confirm" + 0.013*"cure" + 0.011*"donald"
Topic: 5 
Words: 0.052*"coronaviru" + 0.028*"covid" + 0.020*"peopl" + 0.017*"claim" + 0.012

**Running LDA using TF-IDF**

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.017*"coronaviru" + 0.012*"die" + 0.011*"covid" + 0.010*"mask" + 0.009*"peopl" + 0.009*"case" + 0.009*"wear" + 0.007*"claim" + 0.007*"kill" + 0.007*"help"
Topic: 1 Word: 0.017*"test" + 0.016*"cure" + 0.013*"coronaviru" + 0.012*"covid" + 0.011*"case" + 0.010*"total" + 0.008*"sampl" + 0.008*"death" + 0.008*"number" + 0.007*"confirm"
Topic: 2 Word: 0.025*"case" + 0.018*"state" + 0.014*"covid" + 0.014*"report" + 0.013*"test" + 0.011*"total" + 0.011*"confirm" + 0.011*"updat" + 0.008*"daili" + 0.008*"coronaviru"
Topic: 3 Word: 0.020*"coronaviru" + 0.014*"case" + 0.014*"covid" + 0.009*"death" + 0.008*"confirm" + 0.008*"test" + 0.008*"lago" + 0.008*"nigeria" + 0.007*"say" + 0.007*"vaccin"
Topic: 4 Word: 0.019*"test" + 0.016*"report" + 0.015*"number" + 0.013*"death" + 0.012*"state" + 0.012*"complet" + 0.010*"case" + 0.010*"yesterday" + 0.009*"covid" + 0.009*"total"
Topic: 5 Word: 0.016*"test" + 0.013*"covid" + 0.009*"case" + 0.009*"coronaviru" + 0.008*"manag" + 0.008*"china" + 0

**Performance evaluation by classifying sample document using LDA Bag of Words model**

In [ ]:
#processed_docs[4310]
processed_docs[10699]

['coronaviru',
 'case',
 'confirm',
 'highest',
 'daili',
 'number',
 'case',
 'report',
 'friday',
 'overal',
 'total',
 'nationwid',
 'stand',
 'read',
 'latest',
 'http',
 'tydorz',
 'http']

In [ ]:
#for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

for index, score in sorted(lda_model[bow_corpus[10699]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9399852752685547	 
Topic: 0.067*"test" + 0.048*"report" + 0.034*"case" + 0.032*"state" + 0.028*"covid" + 0.023*"death" + 0.022*"number" + 0.019*"daili" + 0.018*"updat" + 0.017*"coronaviru"


**Performance evaluation by classifying sample document using LDA TF-IDF model.**

In [ ]:
#for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

for index, score in sorted(lda_model_tfidf[bow_corpus[10699]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4978351294994354	 
Topic: 0.020*"coronaviru" + 0.014*"case" + 0.014*"covid" + 0.009*"death" + 0.008*"confirm" + 0.008*"test" + 0.008*"lago" + 0.008*"nigeria" + 0.007*"say" + 0.007*"vaccin"

Score: 0.44881942868232727	 
Topic: 0.025*"case" + 0.018*"state" + 0.014*"covid" + 0.014*"report" + 0.013*"test" + 0.011*"total" + 0.011*"confirm" + 0.011*"updat" + 0.008*"daili" + 0.008*"coronaviru"


**Testing model on unseen document**

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6999836564064026	 Topic: 0.044*"covid" + 0.020*"health" + 0.019*"state" + 0.017*"coronaviru" + 0.013*"case"
Score: 0.03333917632699013	 Topic: 0.052*"coronaviru" + 0.028*"covid" + 0.020*"peopl" + 0.017*"claim" + 0.012*"govern"
Score: 0.03333699330687523	 Topic: 0.076*"coronaviru" + 0.032*"case" + 0.031*"test" + 0.030*"trump" + 0.027*"covid"
Score: 0.03333697468042374	 Topic: 0.092*"covid" + 0.020*"test" + 0.018*"state" + 0.016*"patient" + 0.014*"hospit"
Score: 0.03333413228392601	 Topic: 0.038*"covid" + 0.028*"coronaviru" + 0.025*"recoveri" + 0.022*"mask" + 0.020*"say"
Score: 0.03333406522870064	 Topic: 0.053*"covid" + 0.035*"test" + 0.019*"state" + 0.018*"coronaviru" + 0.015*"peopl"
Score: 0.03333393856883049	 Topic: 0.065*"covid" + 0.044*"coronaviru" + 0.030*"vaccin" + 0.015*"pandem" + 0.014*"peopl"
Score: 0.033333856612443924	 Topic: 0.067*"test" + 0.048*"report" + 0.034*"case" + 0.032*"state" + 0.028*"covid"
Score: 0.0333336777985096	 Topic: 0.104*"case" + 0.066*"covid" + 